In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error
import pandas as pd
import numpy as np
from itertools import product
import math
import time
import json
import os
import matplotlib
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from pathlib import Path

from helper_functions import utils
from IPython.display import display
import dataframe_image as dfi
import joblib # for save pipeline

# for display plot inline
%matplotlib inline
# change the style
matplotlib.style.use('ggplot')


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')

Using device: cuda


# Load Data

In [2]:
# Load your dataset

dataset_f = '../data/dataset/2023_Residential_extra_preprocessed.csv'
metadata_f = '../data/dataset/2023_Residential_extra_preprocessed.json'

# This is for saving the prediciton results
output_dir = Path(f'./results/mlp/2023_Residential_extra_preprocessed_gridsearch/')
output_dir.mkdir(parents=True, exist_ok=True)

figure_name = "MLP"
# figure_name = "XGBoost<br>(no extra)"

remove_features = [
    # estimation
    # "Lp_dol", "Taxes",
    # school
    # "s_2km_total", "s_2km_public", "s_2km_private", "s_2km_catholic", "s_2km_ele", "s_2km_sec", "s_2km_hi_ele_score", "s_2km_hi_sec_score", "s_2km_avg_ele_score", "s_2km_avg_sec_score",
    # # census
    # "labour_force_perc", "avg_household_income", "owned_perc", "with_children_perc", "households", "married__perc", "single__perc", "university_degree_perc", "median_age", "population", "age_0_9_perc", "age_10_19_perc", "age_20_34_perc", "age_35_64_perc",
    # # market
    # "average_sold_price", "average_listing_price", "median_sold_price", "median_listing_price", "average_days_on_market", "median_days_on_market", "dollar_volume", "new_listing_count", "sold_listing_count", "sold_overasking_count", "sold_underasking_count", "active_listing_count", "sales_to_new_listing_ratio", "months_of_inventory",
    # # transit
    # "ttc_subway_1km", "go_train_1km", "trainsit_1km",
]

dataset = pd.read_csv(dataset_f)
dataset.set_index('Ml_num', inplace = True)

with open(metadata_f, "r") as f:
    metadata = json.loads(f.read())


with open(os.path.join(output_dir, "remove_features.json"), "w") as f:
    f.write(json.dumps(remove_features))

# Handle datatype res

In [3]:
numerics_int = metadata["features"]["integer"]
numerics_float = metadata["features"]["float"]
numerics_bool = metadata["features"]["boolean"]
categories = metadata["features"]["category"]
    

for num in numerics_float:
    dataset[num] = dataset[num].fillna(0).astype(float)
    
for num in numerics_int:
    dataset[num] = dataset[num].fillna(0).round().astype('int64')
    
for num in numerics_bool:
    dataset[num] = dataset[num].astype('long')

for category in categories:
    dataset[category] = dataset[category].astype("category")


features = []
for k,v in metadata["features"].items():
    features.extend(v)


features_set = set(features)
for feat in remove_features:
    if feat in features_set:
        features.remove(feat)

    
dataset = dataset[features]

# Preprocessing pipeline

In [4]:

numerics_int = metadata["features"]["integer"]
numerics_float = metadata["features"]["float"]
numerics_bool = metadata["features"]["boolean"]
categories = metadata["features"]["category"]

target = 'Sp_dol'

numerical_features = numerics_int + numerics_float + numerics_bool
categorical_features = categories

# Just in case the target value in features
numerical_features.remove(target) if target in numerical_features else None



# Preprocessing pipelines for both numeric and categorical data
numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])
categorical_transformer = Pipeline(steps=[('onehot', OneHotEncoder(handle_unknown='ignore'))])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)])




pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
X_full = dataset[numerical_features + categorical_features]
y_full = dataset[target]

# pipeline should only apply on training set, not on validation set
# X = pipeline.fit_transform(X)


In [5]:
def get_results(model, X_, X, y, save_dir):

    # X.set_index('Ml_num', inplace = True)
    
    model.eval()
    with torch.no_grad():

        # X_tensor = torch.tensor(X_val.toarray()).float().to(device)
        # y_tensor = torch.tensor(y_val).float().unsqueeze(1).to(device)
        
        model_pred = model(torch.tensor(X.toarray()).float().to(device))

    model_pred = model_pred.view(-1).cpu().detach().numpy()
    # X_, y_ = X.cpu().detach().numpy(), y.view(-1).cpu().detach().numpy()
    # X_.set_index('Ml_num', inplace = True)
    
    
    utils.predict_result(X_, y, model_pred, os.path.join(save_dir, "validate_result.csv"))
    
    style_worst = utils.display_worst_prediction(X_, y, model_pred, figure_name)
    dfi.export(style_worst, os.path.join(save_dir, 'top_worst_predictions.png'))

    style_pred_area = utils.display_predict_result(X_, y, model_pred, figure_name, group_by="Area", sort_by="Homes", ascending=False)
    dfi.export(style_pred_area, os.path.join(save_dir, 'predictions_by_area.png'))

    style_pred_muni = utils.display_predict_result(X_, y, model_pred, figure_name, group_by="Municipality_district", sort_by="Homes", ascending=False)
    dfi.export(style_pred_muni, os.path.join(save_dir, 'predictions_by_municipality.png'))


class MLPModel(nn.Module):
    def __init__(self, input_dim, layer_sizes, dropout_rates):
        super(MLPModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_dim, layer_sizes[0]),
            nn.ReLU(),
            nn.Dropout(dropout_rates[0]),
            nn.Linear(layer_sizes[0], layer_sizes[1]),
            nn.ReLU(),
            nn.Dropout(dropout_rates[1]),
            nn.Linear(layer_sizes[1], layer_sizes[2]),
            nn.ReLU(),
            nn.Linear(layer_sizes[2], 1)
            # LeakyReLU
        )
        
    def forward(self, x):
        return self.layers(x)


def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs=5, plot_every=100, plot=True, save_path=""):
    model.train()
    train_loss = []
    train_mae = []
    val_mae = []
    train_mse = []
    val_mse = []
    train_rmse = []
    val_rmse = []
    
    for epoch in range(epochs):
        for inputs, targets in train_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            train_loss.append(float(loss))
            loss.backward()
            optimizer.step()
        # train_mae.append(evaluate_model(model, train_loader)[0])
        # val_mae.append(evaluate_model(model, val_loader)[0])
        mae, mse, rmse = evaluate_model(model, train_loader)
        train_mae.append(mae)
        train_mse.append(mse)
        train_rmse.append(rmse)
        mae, mse, rmse = evaluate_model(model, val_loader)
        val_mae.append(mae)
        val_mse.append(mse)
        val_rmse.append(rmse)

    if plot:
        every_n_train_loss = train_loss[0::plot_every]
        fig = plt.figure(figsize=(10, 5))
        plt.plot(list(map(lambda x:x*plot_every, range(1,len(every_n_train_loss)+1))), every_n_train_loss, label='Loss')
        plt.title(f"Loss over Every {plot_every} Iterations")
        plt.xlabel(f"Every {plot_every} Iterations")
        plt.ylabel("Loss")
        plt.savefig(os.path.join(save_path, 'loss.png'))
        plt.close(fig)
        
        # # Plotting the MAE
        fig = plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(train_mae)+1), train_mae, label='Train MAE')
        plt.plot(range(1, len(val_mae)+1), val_mae, label='Val MAE')
        plt.xlabel(f'{epochs} Epochs')
        plt.ylabel('MAE')
        plt.title('Mean Absolute Error Over Epochs')
        plt.legend()
        plt.savefig(os.path.join(save_path, 'mae.png'))
        plt.close(fig)

        # # Plotting the MSE
        fig = plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(train_mse)+1), train_mse, label='Train MSE')
        plt.plot(range(1, len(val_mse)+1), val_mse, label='Val MSE')
        plt.xlabel(f'{epochs} Epochs')
        plt.ylabel('MSE')
        plt.title('Mean Squared Error Over Epochs')
        plt.legend()
        plt.savefig(os.path.join(save_path, 'mse.png'))
        plt.close(fig)


        # # Plotting the RMSE
        fig = plt.figure(figsize=(10, 5))
        plt.plot(range(1, len(train_rmse)+1), train_rmse, label='Train RMSE')
        plt.plot(range(1, len(val_rmse)+1), val_rmse, label='Val RMSE')
        plt.xlabel(f'{epochs} Epochs')
        plt.ylabel('RMSE')
        plt.title('Root Mean Squared Error Over Epochs')
        plt.legend()
        plt.savefig(os.path.join(save_path, 'rmse.png'))
        plt.close(fig)


# Evaluation function to calculate MAE, MSE, and RMSE
def evaluate_model(model, val_loader):
    model.eval()
    predictions, actuals = [], []
    with torch.no_grad():
        for inputs, targets in val_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            predictions.extend(outputs.view(-1).tolist())
            actuals.extend(targets.view(-1).tolist())

    mse = mean_squared_error(actuals, predictions)
    mae = mean_absolute_error(actuals, predictions)
    rmse = math.sqrt(mse)
    return mae, mse, rmse


# Perform grid search with cross-validation over the hyperparameters
def grid_search_cv(hyperparams, X, y, n_splits=5, epochs=10):

    start_time = time.time()
    
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)
    results = []
    print_results = []

    for num_neurons, lr, batch_size, dropout_rates in product(hyperparams['num_neurons'], hyperparams['learning_rate'], hyperparams['batch_size'], hyperparams['dropout_rates']):
        fold_results = []

        save_path = Path(os.path.join(output_dir, f"Neurons {num_neurons}, LR {lr}, Batch {batch_size}, Dropout {dropout_rates} Epochs {epochs}/"))
        save_path.mkdir(parents=True, exist_ok=True)
        
        for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
            X_train_, X_val_ = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y[train_idx], y[val_idx]
            
            # apply transform pipline
            X_train, X_val = pipeline.fit_transform(X_train_), pipeline.transform(X_val_)

            train_dataset = TensorDataset(torch.tensor(X_train.toarray()).float(), torch.tensor(y_train).float().unsqueeze(1))
            val_dataset = TensorDataset(torch.tensor(X_val.toarray()).float(), torch.tensor(y_val).float().unsqueeze(1))
            train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
            val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

            model = MLPModel(input_dim=X_train.shape[1], layer_sizes=num_neurons, dropout_rates=dropout_rates).to(device)
            loss_function = nn.MSELoss() #nn.L1Loss() # nn.MSELoss()
            optimizer = optim.Adam(model.parameters(), lr=lr)

            save_path_fold = Path(os.path.join(save_path, f"Fold {fold}/"))
            save_path_fold.mkdir(parents=True, exist_ok=True)
            
            train_model(model, train_loader, val_loader, loss_function, optimizer, epochs=epochs, save_path=save_path_fold)

            model_scripted = torch.jit.script(model) # Export to TorchScript
            model_scripted.save(os.path.join(save_path_fold, "mlp_model.pt")) # Save
            joblib.dump(pipeline, os.path.join(save_path_fold, 'pipeline.pkl'))
            
            mae, mse, rmse = evaluate_model(model, val_loader)

            get_results(model, dataset.iloc[val_idx], X_val, y_val, save_path_fold)
            
            fold_results.append((mae, mse, rmse))
        
        avg_results = np.mean(fold_results, axis=0)
        results.append((num_neurons, lr, batch_size, dropout_rates, avg_results.tolist()))

        print_result = f"Neurons: {num_neurons}, LR: {lr}, Batch: {batch_size}, Dropout: {dropout_rates} Loss: MSELoss, Optimizer: Adam, Avg MAE: {avg_results[0]:.4f}, Avg MSE: {avg_results[1]:.4f}, Avg RMSE: {avg_results[2]:.4f} Time: {time.time()-start_time:.2f} seconds"
        print_results.append(print_result)
        print(print_result)

    with open(os.path.join(output_dir, "result.json"), "w") as f:
        f.write(json.dumps(results, ensure_ascii=False))

    with open(os.path.join(output_dir, "result.txt"), "w") as f:
        for txt in print_results:
            f.write(txt + "\n")

    # Find the best parameters based on the average RMSE
    best_params = min(results, key=lambda x: x[4][2])[:4]  # Get the params part of the tuple based on RMSE
    return best_params, results


In [6]:
hyperparameters = {
    'num_neurons': [(128, 64, 32), (256, 128, 64)],
    'learning_rate': [0.01, 0.001],
    'batch_size': [64, 128],
    'dropout_rates': [(0.2, 0.2), (0.5, 0.5)]
} # 20 mins: 2x2x2x2

# hyperparameters = {
#     'num_neurons': [(256, 128, 64)],
#     'learning_rate': [0.01],
#     'batch_size': [128],
#     'dropout_rates': [(0.2, 0.2)]
# }
epochs = 10
best_params, results = grid_search_cv(hyperparameters, X_full, y_full, epochs=epochs)
print("Best Params based on RMSE:", best_params)

with open(os.path.join(output_dir, "best_result.json"), "w") as f:
    f.write(json.dumps(best_params, ensure_ascii=False))

Neurons: (128, 64, 32), LR: 0.01, Batch: 64, Dropout: (0.2, 0.2) Loss: MSELoss, Optimizer: Adam, Avg MAE: 74965.7690, Avg MSE: 13704241571.2410, Avg RMSE: 116989.4553 Time: 186.75 seconds
Neurons: (128, 64, 32), LR: 0.01, Batch: 64, Dropout: (0.5, 0.5) Loss: MSELoss, Optimizer: Adam, Avg MAE: 76474.9845, Avg MSE: 14502103435.0723, Avg RMSE: 120241.6843 Time: 371.95 seconds
Neurons: (128, 64, 32), LR: 0.01, Batch: 128, Dropout: (0.2, 0.2) Loss: MSELoss, Optimizer: Adam, Avg MAE: 74401.4551, Avg MSE: 13848495198.7452, Avg RMSE: 117605.5463 Time: 500.97 seconds
Neurons: (128, 64, 32), LR: 0.01, Batch: 128, Dropout: (0.5, 0.5) Loss: MSELoss, Optimizer: Adam, Avg MAE: 74162.4871, Avg MSE: 14049339214.9273, Avg RMSE: 118457.7167 Time: 639.00 seconds
Neurons: (128, 64, 32), LR: 0.001, Batch: 64, Dropout: (0.2, 0.2) Loss: MSELoss, Optimizer: Adam, Avg MAE: 98520.3160, Avg MSE: 33052798933.5439, Avg RMSE: 180570.0567 Time: 826.96 seconds
Neurons: (128, 64, 32), LR: 0.001, Batch: 64, Dropout: (0

In [7]:
start_time = time.time()
# X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val_, y_train, y_val = train_test_split(X_full, y_full, test_size=0.2, shuffle=False)

num_neurons, lr, batch_size, dropout_rates = best_params

# apply transform pipline
X_train, X_val = pipeline.fit_transform(X_train), pipeline.transform(X_val_)

train_dataset = TensorDataset(torch.tensor(X_train.toarray()).float(), torch.tensor(y_train).float().unsqueeze(1))
val_dataset = TensorDataset(torch.tensor(X_val.toarray()).float(), torch.tensor(y_val).float().unsqueeze(1))
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

model = MLPModel(input_dim=X_train.shape[1], layer_sizes=num_neurons, dropout_rates=dropout_rates).to(device)
loss_function = nn.MSELoss() #nn.L1Loss() # nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

save_path = Path(os.path.join(output_dir, f"best_model/"))
save_path.mkdir(parents=True, exist_ok=True)

train_model(model, train_loader, val_loader, loss_function, optimizer, epochs=epochs, save_path=save_path)

model_scripted = torch.jit.script(model) # Export to TorchScript
model_scripted.save(os.path.join(save_path, "mlp_model.pt")) # Save
joblib.dump(pipeline, os.path.join(save_path, 'pipeline.pkl'))

mae, mse, rmse = evaluate_model(model, val_loader)

get_results(model, X_val_, X_val, y_val, save_path)

print_result = f"Neurons: {num_neurons}, LR: {lr}, Batch: {batch_size}, Dropout: {dropout_rates} Loss: MSELoss, Optimizer: Adam, Avg MAE: {mae:.4f}, Avg MSE: {mse:.4f}, Avg RMSE: {rmse:.4f} Time: {time.time()-start_time:.2f} seconds"
print(print_result)

Neurons: (128, 64, 32), LR: 0.01, Batch: 64, Dropout: (0.2, 0.2) Loss: MSELoss, Optimizer: Adam, Avg MAE: 77549.6466, Avg MSE: 15721602378.5215, Avg RMSE: 125385.8141 Time: 36.00 seconds
